In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import pyautogui
import time

base_tok_ia = pd.read_excel("base_de-dados")

options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get(
        "endereço_do_site"
    )
    driver.implicitly_wait(10)

    campo_usuario = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "idToken1"))
    )
    campo_usuario.send_keys("campo_login")

    campo_senha = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "idToken2"))
    )
    campo_senha.send_keys("campo_senha")

    botao_login = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "loginBut..."))
    )
    botao_login.click()

    click_concessionaria = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@href='/group....']"))
    )
    click_concessionaria.click()

    for index, row in base_tok_ia.iterrows():
        cpf_tokio = str(row["..."])
        cod_tokio = str(row["..."])

        try:
            click_add_conc = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//a...']"))
            )

            time.sleep(1)
            click_add_conc.click()

            digitar_cod = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@..."))
            )
            digitar_cod.clear()
            digitar_cod.send_keys(cod_tokio)

            clicar_cod = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "tr..."))
            )
            clicar_cod.click()

            for tentativa in range(3):
                try:
                    click_conta_corrente = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@h..."))
                    )
                    click_conta_corrente.click()
                    break
                except StaleElementReferenceException:
                    pass
            else:
                raise Exception("Falha ao clicar em 'Conta Corrente' após 3 tentativas.")

            consulta_saldo = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//a[c...")
                )
            )
            consulta_saldo.click()

            time.sleep(1)

            x, y = 680, 409
            pyautogui.moveTo(x, y, duration=0.5)
            pyautogui.click()

            time.sleep(2)
            action = ActionChains(driver)
            action.send_keys(cpf_tokio[:3])
            action.send_keys(Keys.RETURN)
            action.perform()

            iframe = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "ifr..."))
            )
            driver.switch_to.frame(iframe)

            saldo_atual_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a..."))
            )
            saldo_atual = saldo_atual_element.text.strip()

            base_tok_ia.at[index, "valor"] = str(saldo_atual)

            driver.switch_to.default_content()

        except TimeoutException:
            pass

except Exception as e:
    print(f"Erro geral: {e}")

finally:
    driver.quit()
    base_tok_ia.to_excel("C:/...", index=False)
